In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from torchsummary import summary
import os
import time

In [15]:
DATA_DIR = 'data/'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
USE_RAW = False

In [16]:
def random_transformation(im):
    """Randomly rotate or flip the image"""
    i = np.random.randint(8)
    if i == 0 :
        return im
    if i == 1 :
        return np.rot90(im, axes=(0,1), k=1)
    if i == 2 :
        return np.rot90(im, axes=(0,1), k=2)
    if i == 3 :
        return np.rot90(im, axes=(0,1), k=3)
    if i == 4:
        return np.flip(im, axis=1)
    if i == 5:
        return np.flip(np.rot90(im, axes=(0,1), k=1))
    if i == 6:
        return np.flip(np.rot90(im, axes=(0,1), k=2))
    if i == 7:
        return np.flip(np.rot90(im, axes=(0,1), k=3))

In [17]:
class HackathonDataset(Dataset):
    """Hackathon Dataset"""   
    
    def __init__(self, csv_file, root_dir, use_raw=False, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with paths and labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.file = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.use_raw = use_raw
        
    def __len__(self):
        return len(self.file)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()        
        img_file = self.file.iloc[idx]["image_file_name"]        
        if "22NCL" in img_file:
            img_dir= "guyane/guyane/"
        elif "28PCC" in img_file:
            img_dir = "saint_louis/saint_louis/"  
        elif "29SMD" in img_file:
            img_dir = "dataset_29SMD/dataset_29SMD/"  
        elif "29TNE" in img_file:
            img_dir = "dataset_29TNE/dataset_29TNE/"
        else:
            raise Exception('There is something wrong with image name')    
        image = np.load(self.root_dir + img_dir + img_file + ".npy")
        if self.use_raw:
            image_raw = np.load(self.root_dir + img_dir + img_file + "_RAW.npy")
            image = np.concatenate((image, image_raw), axis=2)
        if self.transform:
            image = random_transformation(image)  # Add a random permutation of the image
        image = np.moveaxis(image, -1, 0)  # Permute dimensions in order to have Cin, H, W instead of H, W, Cin
        image = image.astype(np.float32)  # We work with float (float32), not double (float64)
        target = self.file.iloc[idx]["z"]
        target = target.astype(np.float32)  # We work with float (float32), not double (float64)
        sample = {'image': image, 'z': target, "image_file_name": img_file}
        return sample

In [29]:
class Net(nn.Module):
    """Used Network"""

    def __init__(self, device, use_raw = USE_RAW):
        self.use_raw = use_raw
        super(Net, self).__init__()
        self.device = device
        self.reset()
    
    def reset(self):
        self.conv1 = nn.Conv2d(8 if self.use_raw else 4, 32, 3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
#         self.conv3 = nn.Conv2d(32, 256, 3, padding=1)
#         self.relu3 = nn.ReLU()
#         self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(6400, 128)  
        self.relu4 = nn.ReLU()
#         self.fc2 = nn.Linear(1024, 1024)
#         self.relu5 = nn.ReLU()
        self.fc3 = nn.Linear(128, 1)
        self.relu6 = nn.ReLU()
        self.to(self.device)

    def forward(self, x):
        """Given a tensor X of shape (Batch_size, C_in, H, W), compute the output tensor, of shape (Batch_size, )"""
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
#         x = self.conv3(x)
#         x = self.relu3(x)
#         x = self.pool3(x)
        x = x.flatten(start_dim=1)  # Flatten the 3 last dimensions, keep the 1 dimension (batch_size)
        x = self.fc1(x)
        x = self.relu4(x)
#         x = self.fc2(x)
#         x = self.relu5(x)
        x = self.fc3(x)
        x = self.relu6(x)
        x = x.flatten()  # The output dimension should be (Batch_size, ) and not (Batch_size, 1) 
        return x
    
    def fit(self, train_dataloader, test_dataloader, n_epochs, optimizer=None, criterion=nn.MSELoss()):
        
        while True:
        
            if optimizer is None:
                optimizer = torch.optim.Adam(self.parameters())
            
            print_frequency = 2
            train_losses = []
            test_losses = []
            
            for epoch in range(n_epochs):
                
                timer = time.time()
                
                # Print Epoch
                print(f"Epoch {epoch + 1}/{n_epochs}")
                
                # Training loop
                for it, batch in enumerate(train_dataloader):
                            
                    # Reset gradients
                    optimizer.zero_grad()
                    
                    # Forward propagation through the network
                    out = self(batch["image"].to(DEVICE))

                    # Calculate the loss
                    loss = torch.sqrt(criterion(out, batch["z"].to(DEVICE)))  # We take square root because RMSE is the competition's metric

                    # Track batch loss
                    train_losses.append(loss.item())

                    # Backpropagation
                    loss.backward()

                    # Update the parameters
                    optimizer.step()

                    #=====Printing part======
                    if (it+1)%(len(train_dataloader) // print_frequency) == 0:
                        print(f"Number of sample viewed : {it*batch_size}")
                        print(f"Current training loss : {np.mean(train_losses[-len(train_dataloader)//print_frequency:-1])}")

                        # Validation loop
                        for it, batch in enumerate(test_dataloader):

                            # Forward propagation through the network
                            out = self(batch["image"].to(DEVICE))

                            # Calculate the loss
                            loss = torch.sqrt(criterion(out, batch["z"].to(DEVICE)))  # We take square root because RMSE is the competition's metric

                            # Track batch loss
                            test_losses.append(loss.item())
            
                        print(f"Current validation loss : {np.mean(test_losses[-int(len(test_dataloader)*0.8):-1])}")
                
                print(f"The epoch took {time.time() - timer: .2f} seconds")
            if np.mean(train_losses) < 15:
                break
            
            self.reset()
            
                
    def predict(self, dataloader):
        predictions = []
        for it, batch in enumerate(dataloader):
            out = list(self(batch["image"].to(DEVICE)).cpu().detach().numpy())
            predictions += out
        return predictions

In [30]:
class Ensemble:
    
    def __init__(self, Model, device, n_estimators):
        self.Model = Model
        self.instances = [self.Model(device) for i in range(n_estimators)]
    
    def fit(self, train_dataloader, test_dataloader, n_epochs):
        for it, instance in enumerate(self.instances):
            print(f"=== Training instance {it+1}/{len(self.instances)} ===")
            instance.fit(train_dataloader, test_dataloader, n_epochs)
    
    def predict(self, dataloader):
        predictions = [instance.predict(dataloader) for instance in self.instances]
        return np.mean(predictions, axis=0)

In [37]:
n_epochs = 3
n_estimators = 5
batch_size = 16  # High batch size often happen to not converge... So we use small batches, even if slower
pred_batch_size = 128  # There is no problem of convergence for training batch size

In [38]:
#========================NOTE============================
# We often have to reset the model, because it won't converge. I don't know why, but it is useful to know
# If the training loss is stuck around 22 and the validation loss is stuck around 10,
# reset the model by running this cell again, and relaunch training
#========================END Of NOTE=====================

dataset = HackathonDataset(DATA_DIR + 'mixed_train.csv', DATA_DIR, USE_RAW)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count() - 2)
val_dataset = HackathonDataset(DATA_DIR + 'mixed_validation.csv', DATA_DIR, USE_RAW)
val_dataloader = DataLoader(val_dataset, batch_size=pred_batch_size, shuffle=False, num_workers=os.cpu_count() - 2)
model = Ensemble(Net, DEVICE, n_estimators)

In [39]:
model.fit(dataloader, val_dataloader, n_epochs)

=== Training instance 1/5 ===
Epoch 1/3
Number of sample viewed : 28480
Current training loss : 7.3379768899317535
Current validation loss : 5.502696065452155
Number of sample viewed : 56976
Current training loss : 6.542136395111513
Current validation loss : 4.1068934174034535
The epoch took  14.97 seconds
Epoch 2/3
Number of sample viewed : 28480
Current training loss : 6.1871063851238635
Current validation loss : 5.513519717013742
Number of sample viewed : 56976
Current training loss : 5.832306250829375
Current validation loss : 3.979257293573515
The epoch took  16.31 seconds
Epoch 3/3
Number of sample viewed : 28480
Current training loss : 5.675735554467426
Current validation loss : 5.1043099202509
Number of sample viewed : 56976
Current training loss : 5.556661059548346
Current validation loss : 3.63818284094803
The epoch took  16.81 seconds
=== Training instance 2/5 ===
Epoch 1/3
Number of sample viewed : 28480
Current training loss : 7.384340753046314
Current validation loss : 5.

In [40]:
def smoothed_curves(tab, gamma=0.99):
    ret = [tab[min(len(tab)-1, 2000)]]
    for val in tab:
        ret.append(ret[-1]*gamma + val * (1-gamma))
    return ret

In [41]:
#plt.plot(smoothed_curves(train_losses))
#plt.show()
#plt.plot(smoothed_curves(val_losses))
#plt.show()

# Evaluation on Test Data

In [42]:
test_dataset = HackathonDataset(DATA_DIR + 'mixed_test.csv', DATA_DIR, USE_RAW)
test_dataloader = DataLoader(test_dataset, batch_size=pred_batch_size, shuffle=False, num_workers=os.cpu_count() - 2)

In [43]:
image_file_names = []
for val in test_dataloader:
    image_file_names += val['image_file_name']

predictions = model.predict(test_dataloader)
kaggle_df = pd.DataFrame({'image_id': image_file_names,
                          'predicted_z': predictions})

In [44]:
kaggle_df.to_csv('predictions/prediction-' + datetime.now().strftime("%d-%m-%y:%H-%M") + '.csv', index=False)